In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.optim as optim

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [2]:
import numpy as np

In [3]:
train_dataset = dsets.MNIST(root='./MNIST/', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./MNIST/', 
                           train=False, 
                           transform=transforms.ToTensor())

In [286]:
class ActNorm(nn.Module):
    def __init__(self, h, w, c):
        super().__init__()
        self.shape = (h, w, c)
        self.initialized = False
        self.weights = nn.Parameter(torch.Tensor(c))
        self.bias = nn.Parameter(torch.Tensor(c))
#         self.weights.data.uniform_(-0.1, 0.1)
#         self.bias.data.uniform_(-0.1, 0.1)
        
    def forward(self, inp, logdet):
        if not self.initialized:
            c = self.shape[-1]
            self.weights.data = 1/inp.view(-1, c).std(0)
            self.bias.data = -(inp * self.weights).view(-1, c).mean(0)
            self.initialized = True
        
        return inp * self.weights + self.bias, logdet + self.shape[0] * self.shape[1] * torch.log(torch.abs(self.weights))
    
    def reverse(self, out):
        return (out - self.bias) / self.weights

class InvertibleConv(nn.Module):
    def __init__(self, h, w, c):
        super().__init__()
        self.shape = (h, w, c)
        self.weight = nn.Parameter(torch.from_numpy(np.linalg.qr(np.random.randn(c, c))[0]))
                                    
    def forward(self, inp, logdet):
        return torch.einsum("abcd,eb->aecd", (inp, self.weight)), logdet + self.shape[0] * self.shape[1] * torch.log(torch.abs(torch.det(self.weight)))
    
    def reverse(self, out):
        return torch.einsum("abcd,eb->aecd", (inp, torch.inverse(self.weight)))

class AffineCoupling(nn.Module):
    def __init__(self, NN):
        super().__init__()
        self.NN = NN
        
    def forward(self, inp, logdet):
        x_a, x_b = torch.split(inp, inp.shape[2] // 2, dim = 2)
        log_s, t = self.NN(x_b)
        s = torch.exp(log_s)
        y_a = s * x_a + t
        return torch.cat([y_a, x_b], dim = 2), logdet + torch.sum(torch.log(torch.abs(s)))
    
    def reverse(self, out):
        y_a, y_b = torch.split(out, out.shape[2] // 2, dim = 2)
        log_s, t = self.NN(y_b)
        s = torch.exp(log_s)
        x_a = (y_a - t) / s
        return torch.cat([x_a, y_b], dim = 2)
    
class Flow(nn.Module):
    def __init__(self, K, NN, h, w, c):
        super().__init__()
        self.k = K
        self.actnorms = [ActNorm(h, w, c) for i in range(K)]
        self.invconvs = [InvertibleConv(h, w, c) for i in range(K)]
        self.couplings = [AffineCoupling(NN) for i in range(K)]
        
    def forward(self, inp, logdet, z):
        inp = inp.view(inp.shape[0] / 2, int.shape[1] / 2, -1)
        
        for i in range(self.k):
            out, logdet = self.actnorm[i](inp, logdet)
            out, logdet = self.invconv[i](out, logdet)
            out, logdet = self.coupling[i](out, logdet)
        
        out, z_i = torch.split(out, out.shape[2] // 2, dim = 2)
        z = torch.cat([z, z_i.view(-1)])
        
        return out, logdet, z
    
    def reverse(self, out):
        out = self.actnorm.reverse(out)
        out = self.invconv.reverse(out)
        out = self.coupling.reverse(out)
        return out

In [355]:
def pixel_unshuffle(input_, block_size=2):
    b, c, h, w = input_.shape

    assert h % block_size == 0 and w % block_size == 0,\
        f"Shape must be divisible by block_size, got {input_.shape}"

    oc = c * block_size * block_size;
    oh = h // block_size;
    ow = w // block_size;

    input_reshaped = input_.view(b, c, oh, block_size, ow, block_size)
    return input_reshaped.permute(0, 1, 3, 5, 2, 4).reshape(b, oc, oh, ow)
    
class Squeeze(nn.Module):
    def __init__(self, block_size=2):
        super().__init__()
        self.block_size = block_size 

    def forward(self, input_, logdet, z):
        return pixel_unshuffle(input_, self.block_size), logdet, z
    
    def reverse(self, input_, z):
        return F.pixel_shuffle(input_, self.block_size), z

In [356]:
class Split(nn.Module):
    def forward(self, input_, logdet, z):
        out, zi = torch.split(input_, input_.shape[1]//2, 1)

        return out, logdet, z + [zi]
    
    def reverse(self, input_, z):
        out = torch.cat([input_, z[-1]], 1)
        
        return out, z[:-1]

In [6]:
class GlowModel(nn.Module):
    def __init__(self, NN, h, w, c, K, L):
        super().__init__()
        self.flows = [[Flow(NN, h, w, int(c / 2 ** j)) for i in range(K)] for j in range(L)]
        self.k = K
        self.l = L
        
    def forward(self, inp):
        